In [1]:
import pandas as pd
import os

import tkinter as tk
from tkinter import filedialog
import re
import time
import numpy as np


In [2]:
print('正在打开……')
df_mr=pd.DataFrame()
root = tk.Tk()
root.withdraw()
# path = filedialog.askopenfilename()
# path_out, temp = os.path.split(path)
# file_name, extension = os.path.splitext(temp)
# print('path:', path_out)
# print('file_name:', file_name)
# print('extension:', extension)

directory_path = filedialog.askdirectory()

正在打开……


In [3]:
date_now = time.strftime("%Y%m", time.localtime())
# date=int(date)-1 #上个月YYYYMM
date_last=int(date_now[0:4])-1 #上个月YYYYMM
date_last=str(date_last)
date_last=date_last+date_now[4:6]

In [121]:
date_last

'202202'

In [4]:
s1=date_last[4:6]
s2=date_now[4:6]
if s1[0:1]=='0':
    s1=date_last[5:6]
if s2[0:1]=='0':
    s2=date_now[5:6]


datapath1=directory_path+'\\'+date_last[0:4]+'年'+s1+'月科目明细表.xlsx'
datapath2=directory_path+'\\'+date_now[0:4]+'年'+s2+'月科目明细表.xlsx'

In [5]:
df1=pd.read_excel(datapath1,skiprows=1)
df2=pd.read_excel(datapath2,skiprows=1)

In [6]:
df_res_l=df1[(df1["科目段"].astype(str).str.contains('5111'))&(df1["来源"]=="BOSS_IMPORT")&(df1["日记账分录"].astype(str).str.contains('结算'))&(~df1["科目段说明"].astype(str).str.contains('网内|网间|折扣'))]
df_res_n=df2[(df2["科目段"].astype(str).str.contains('5111'))&(df2["来源"]=="BOSS_IMPORT")&(df2["日记账分录"].astype(str).str.contains('结算'))&(~df2["科目段说明"].astype(str).str.contains('网内|网间|折扣'))]

透视表

In [7]:

df_pivot1 = pd.pivot_table(df_res_l.fillna(0),index=['科目段说明','子目段说明','GL期间'], values=['本币借项','本币贷项'],aggfunc={'本币借项':np.sum,'本币贷项':np.sum},margins=True,fill_value=0)

df_pivot1 = df_pivot1.reset_index()

df_pivot2 = pd.pivot_table(df_res_n.fillna(0),index=['科目段说明','子目段说明','GL期间'], values=['本币借项','本币贷项'],aggfunc={'本币借项':np.sum,'本币贷项':np.sum},margins=True,fill_value=0)

df_pivot2 = df_pivot2.reset_index()

### **数据处理**


将科目合并到对应科目表的科目里，分组汇总

In [8]:
key_words=['其他数据业务','咪咕','和教育','手机上网','梦网业务','移动气象站','9%','云计算业务']
merge_key_words=['和生活','和动漫','和游戏','和视频']
other_key_words=key_words+merge_key_words
ss=''
for w in other_key_words:
    ss=ss+'|'+w
ss = ss[1:]
other_key_words=ss

In [9]:
def tmp_merge(df_handle_res3,col_name):
    df_handle_res3=pd.pivot_table(df_handle_res3.fillna(0),index=['GL期间'], values=['本币借项','本币贷项'],aggfunc={'本币借项':np.sum,'本币贷项':np.sum},fill_value=0)
    df_handle_res3=df_handle_res3.reset_index()
    df_handle_res3['子目段说明']=col_name
    df_handle_res3['科目段说明']='数据业务'
    # '数据业务-和生活、和动漫、和游戏、和视频'
    df_handle_res3 = df_handle_res3.reindex(columns=['科目段说明', '子目段说明', 'GL期间', '本币借项', '本币贷项'])
    return df_handle_res3

def handle_project(df_handle_res1,df_handle_res2,df_pivot1,df_pivot2,key_words):

    for key_word in key_words:
        
        df_mg1=df_pivot1[df_pivot1['子目段说明'].str.contains(key_word)]
        df_mg2=df_pivot2[df_pivot2['子目段说明'].str.contains(key_word)]
        if key_word=='和教育' or key_word=='云计算业务':
            df_mg1['类型']='政企'
            df_mg2['类型']='政企'
        else:
            df_mg1['类型']='市场'
            df_mg2['类型']='市场'
        df_handle_res1=pd.concat([df_handle_res1,df_mg1])
        df_handle_res2=pd.concat([df_handle_res2,df_mg2])
    return df_handle_res1,df_handle_res2

def handle_merge_project(df_handle_res3,df_handle_res4,df_pivot1,df_pivot2,merge_key_words):

    for key_word in merge_key_words:
        
        df_mg1=df_pivot1[df_pivot1['子目段说明'].str.contains(key_word)]
        df_mg2=df_pivot2[df_pivot2['子目段说明'].str.contains(key_word)]

        df_handle_res3=pd.concat([df_handle_res3,df_mg1])
        df_handle_res4=pd.concat([df_handle_res4,df_mg2])
    col_name='数据业务-和生活、和动漫、和游戏、和视频'
    df_handle_res3=tmp_merge(df_handle_res3,col_name)
    df_handle_res4=tmp_merge(df_handle_res4,col_name)
    df_handle_res3['类型']='市场'
    df_handle_res4['类型']='市场'
    return df_handle_res3,df_handle_res4

def handle_other_project(df_pivot1,df_pivot2,other_key_words):


    df_mg1=df_pivot1[~df_pivot1['子目段说明'].str.contains(other_key_words)]
    df_mg2=df_pivot2[~df_pivot2['子目段说明'].str.contains(other_key_words)]
    col_name='其他项目'
    df_mg1=tmp_merge(df_mg1,col_name)
    df_mg2=tmp_merge(df_mg2,col_name)
    df_mg1['类型']='市场'
    df_mg2['类型']='市场'
    return df_mg1,df_mg2

In [10]:
df_handle_res1=pd.DataFrame()
df_handle_res2=pd.DataFrame()
df_handle_res3=pd.DataFrame()
df_handle_res4=pd.DataFrame()
df_handle_res5=pd.DataFrame()
df_handle_res6=pd.DataFrame()

In [11]:
%%capture --no-display
df_handle_res1,df_handle_res2=handle_project(df_handle_res1,df_handle_res2,df_pivot1,df_pivot2,key_words)
df_handle_res3,df_handle_res4=handle_merge_project(df_handle_res3,df_handle_res4,df_pivot1,df_pivot2,merge_key_words)
df_handle_res5,df_handle_res6=handle_other_project(df_pivot1,df_pivot2,other_key_words)

In [12]:
df_res_now=pd.concat([df_handle_res2,df_handle_res4,df_handle_res6])
df_res_now

,科目段说明,子目段说明,GL期间,本币借项,本币贷项,类型
5,通信与信息服务收入-应用及信息服务-数据业务,通信服务-数据业务-其他数据业务,2023-01,2156974.14,0.00,市场
12,通信与信息服务收入-应用及信息服务-数据业务,通信服务-数据业务-咪咕音乐（信息费）,2023-01,1449160.02,9830.18,市场
7,通信与信息服务收入-应用及信息服务-数据业务,通信服务-数据业务-和教育,2023-01,1036747.85,0.00,政企
19,通信与信息服务收入-数据流量业务,通信服务-数据流量业务-通服收入用-手机上网（非统付）,2023-01,317738.83,-116.24,市场
16,通信与信息服务收入-应用及信息服务-数据业务,通信服务-数据业务-梦网业务,2023-01,183612.79,0.00,市场
17,通信与信息服务收入-应用及信息服务-数据业务,通信服务-数据业务-移动气象站,2023-01,58032.70,0.00,市场
0,通信与信息服务收入-其他-其他,税率税项-增值税-税率-9%基础电信服务,2023-01,0.00,1325.47,市场
4,通信与信息服务收入-应用及信息服务-数据业务,通信服务-数据业务-云计算业务,2023-01,-15629.13,0.00,政企
0,数据业务,数据业务-和生活、和动漫、和游戏、和视频,2023-01,360698.82,266.07,市场
0,数据业务,其他项目,,5783472.46,22722.01,市场


In [13]:
df_res_last=pd.concat([df_handle_res1,df_handle_res3,df_handle_res5])
df_res_last

,科目段说明,子目段说明,GL期间,本币借项,本币贷项,类型
3,通信与信息服务收入-应用及信息服务-数据业务,通信服务-数据业务-其他数据业务,2022-01,1290967.26,0.00,市场
11,通信与信息服务收入-应用及信息服务-数据业务,通信服务-数据业务-咪咕音乐（信息费）,2022-01,749831.17,-17376.69,市场
6,通信与信息服务收入-应用及信息服务-数据业务,通信服务-数据业务-和教育,2022-01,338939.32,0.00,政企
16,通信与信息服务收入-数据流量业务,通信服务-数据流量业务-通服收入用-手机上网（非统付）,2022-01,-16370.62,-347.91,市场
14,通信与信息服务收入-应用及信息服务-数据业务,通信服务-数据业务-梦网业务,2022-01,232308.37,0.00,市场
15,通信与信息服务收入-应用及信息服务-数据业务,通信服务-数据业务-移动气象站,2022-01,68625.19,0.00,市场
1,通信与信息服务收入-应用及信息服务-数据业务,通信服务-数据业务-云计算业务,2022-01,37481.34,0.00,政企
0,数据业务,数据业务-和生活、和动漫、和游戏、和视频,2022-01,82902.77,141.94,市场
0,数据业务,其他项目,,2883315.09,-9753.93,市场
1,数据业务,其他项目,2022-01,98630.29,7828.73,市场


通信服务-数据业务-其他数据业务

In [24]:
df_data_l=df_res_l[df_res_l['子目段说明']=='通信服务-数据业务-其他数据业务']
df_data_n=df_res_n[df_res_n['子目段说明']=='通信服务-数据业务-其他数据业务']

In [45]:
def res_handle(df_res_last,df_res_l,df_data_l):
        key_dict=['和办公','权益','和交通|警务通','和办公|权益|和交通|警务通']
        df_res_data_l=pd.DataFrame()
        for key in key_dict:
            if len(key)>12:
                df_data=df_data_l[~(df_res_l['行说明'].str.contains(key))]
                df_tmp=df_data[['科目段说明', '子目段说明', '行说明','GL期间', '本币借项', '本币贷项']].fillna(0)
                df_tmp=pd.pivot_table(df_tmp,index=['科目段说明','子目段说明','GL期间'], values=['本币借项','本币贷项'],aggfunc={'本币借项':np.sum,'本币贷项':np.sum},margins=False,fill_value=0)
                df_tmp=df_tmp.reset_index()
                df_tmp['科目']='其他数据业务-'+'其他小额sp'
            else:
                df_data=df_data_l[(df_res_l['行说明'].str.contains(key))]
                df_tmp=df_data[['科目段说明', '子目段说明', '行说明','GL期间', '本币借项', '本币贷项']].fillna(0)
                df_tmp=pd.pivot_table(df_tmp,index=['科目段说明','子目段说明','GL期间'], values=['本币借项','本币贷项'],aggfunc={'本币借项':np.sum,'本币贷项':np.sum},margins=False,fill_value=0)
                df_tmp=df_tmp.reset_index()
                df_tmp['科目']='其他数据业务-'+key
            # print(df_data)
            if key=='和办公' or key=='和交通|警务通':
                df_tmp['类型']='政企'
            else:
                df_tmp['类型']='市场'
            df_res_data_l=pd.concat([df_res_data_l,df_tmp])
        # df_data_l[(df_res_l['行说明'].str.contains('权益'))]
        # df_data_l[(df_res_l['行说明'].str.contains('和交通|警务通'))]
        # df_data_l[(~df_res_l['行说明'].str.contains('和办公|权益|和交通|警务通'))]
        df_res_data_l=df_res_data_l.reset_index()
        df_res_data_l.reindex(columns=['index','科目段说明', '子目段说明', '科目','GL期间', '本币借项', '本币贷项'])
        df_res_data_l['子目段说明']=df_res_data_l['科目']
        del df_res_data_l['index']
        del df_res_data_l['科目']
        ####TODO
        df_res_last=df_res_last[~df_res_last['子目段说明'].str.contains('其他数据业务')]


        df_res1=pd.concat([df_res_last,df_res_data_l])
        return df_res1


In [68]:
df_res1=res_handle(df_res_last,df_res_l,df_data_l)
df_res1=df_res1[~(df_res1['GL期间']=='')]

df_res2=res_handle(df_res_now,df_res_n,df_data_n)
df_res2=df_res2[~(df_res2['GL期间']=='')]

C:\Users\Administrator\AppData\Local\Temp\ipykernel_9088\2857416920.py:12: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  df_data=df_data_l[(df_res_l['行说明'].str.contains(key))]
C:\Users\Administrator\AppData\Local\Temp\ipykernel_9088\2857416920.py:12: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  df_data=df_data_l[(df_res_l['行说明'].str.contains(key))]
C:\Users\Administrator\AppData\Local\Temp\ipykernel_9088\2857416920.py:12: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  df_data=df_data_l[(df_res_l['行说明'].str.contains(key))]
C:\Users\Administrator\AppData\Local\Temp\ipykernel_9088\2857416920.py:6: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  df_data=df_data_l[~(df_res_l['行说明'].str.contains(key))]
C:\Users\Administrator\AppData\Local\Temp\ipykernel_9088\2857416920.py:12: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  df_data=df_data

In [69]:
df_res2

,科目段说明,子目段说明,GL期间,本币借项,本币贷项,类型
12,通信与信息服务收入-应用及信息服务-数据业务,通信服务-数据业务-咪咕音乐（信息费）,2023-01,1449160.02,9830.18,市场
7,通信与信息服务收入-应用及信息服务-数据业务,通信服务-数据业务-和教育,2023-01,1036747.85,0.00,政企
19,通信与信息服务收入-数据流量业务,通信服务-数据流量业务-通服收入用-手机上网（非统付）,2023-01,317738.83,-116.24,市场
16,通信与信息服务收入-应用及信息服务-数据业务,通信服务-数据业务-梦网业务,2023-01,183612.79,0.00,市场
17,通信与信息服务收入-应用及信息服务-数据业务,通信服务-数据业务-移动气象站,2023-01,58032.70,0.00,市场
0,通信与信息服务收入-其他-其他,税率税项-增值税-税率-9%基础电信服务,2023-01,0.00,1325.47,市场
4,通信与信息服务收入-应用及信息服务-数据业务,通信服务-数据业务-云计算业务,2023-01,-15629.13,0.00,政企
0,数据业务,数据业务-和生活、和动漫、和游戏、和视频,2023-01,360698.82,266.07,市场
1,数据业务,其他项目,2023-01,236136.44,11416.53,市场
0,通信与信息服务收入-应用及信息服务-数据业务,其他数据业务-和办公,2023-01,1538212.88,0.00,政企


In [84]:
outer = pd.merge(df_res2, df_res1, on='子目段说明')

In [91]:
outer=outer[['类型_x','子目段说明','本币借项_y','本币贷项_y','本币借项_x','本币贷项_x']]
outer['l_净收入']=outer['本币贷项_y']-outer['本币借项_y']
outer['n_净收入']=outer['本币贷项_x']-outer['本币借项_x']
outer=outer.reindex(columns=['类型_x','子目段说明','本币借项_y','本币贷项_y','l_净收入','本币借项_x','本币贷项_x','n_净收入'])
outer['净增收']=outer['n_净收入']-outer['l_净收入']
outer=outer.set_index(['类型_x','子目段说明'])


C:\Users\Administrator\AppData\Local\Temp\ipykernel_9088\3384332181.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  outer['l_净收入']=outer['本币贷项_y']-outer['本币借项_y']
C:\Users\Administrator\AppData\Local\Temp\ipykernel_9088\3384332181.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  outer['n_净收入']=outer['本币贷项_x']-outer['本币借项_x']


In [99]:
outer=outer.div(10000).round(1)

In [102]:
outer.reset_index().to_excel(directory_path+'/分析基础数据.xlsx',index=0)